# Local Inference with MONet Bundle

## Requirements

- Python 3.10 or higher
- NVidia GPU with CUDA support
- PyTorch 2.4.1 with CUDA support

In [ ]:
%%bash

pip install light-the-torch
ltt install torch==2.4.1 torchvision

In [ ]:
pip install mlflow
pip install git+https://github.com/SimoneBendazzoli93/dynamic-network-architectures.git
pip install git+https://github.com/SimoneBendazzoli93/nnUNet.git
pip install batchgenerators==0.25
pip install git+https://github.com/SimoneBendazzoli93/MONAI.git@dev
pip install nvflare
pip install lightning

In [ ]:
import requests
import json
import torch
from monai.bundle import ConfigParser
from monai.transforms import LoadImaged, EnsureChannelFirstd, Compose, SaveImage

## Download pre-trained MONet bundle

In [ ]:
response = requests.get("https://<MONAI-LABEL_SERVER_URL>/model/MONetBundle")

In [ ]:
if response.status_code == 200:
    with open("MONetBundle.ts", "wb") as f:
        f.write(response.content)
    print("File saved as MONetBundle.ts")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

## Create MONet Predictor

In [ ]:
extra_files = {"inference.json": "","metadata.json": ""}
model = torch.jit.load("MONetBundle.ts",_extra_files=extra_files)

inference = json.loads(extra_files["inference.json"])
metadata = json.loads(extra_files["metadata.json"])

parser = ConfigParser(inference)

nnunet_predictor = parser.get_parsed_content("network_def",instantiate=True)
nnunet_predictor.predictor.network = model

## Load the input image

In [ ]:
transform = Compose([
    LoadImaged(keys=["image"]),
    EnsureChannelFirstd(keys=["image"])
])

In [ ]:
input_image_path = "<INPUT_IMAGE_PATH>"  # Replace with the path to your input image
data = transform({"image": input_image_path})

## Run inference

In [ ]:
pred = nnunet_predictor(data["image"][None])

## Save prediction

In [ ]:
output_folder = "<OUTPUT_FOLDER>"  # Replace with the desired output folder path
pred = SaveImage(output_dir=output_folder,separate_folder=False, output_postfix="segmentation", output_ext=".nii.gz")(pred[0])